In [1]:
import pandas as pd
import folium
import geopandas as gpd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

%cd C:\emobility

C:\emobility


### Convert SySMo's activity_plans to OTP input data format

In [8]:
df_act = pd.read_csv(f'./dbs/agents/1_activity_plans.csv')


0.50    10.256676
0.60    14.195510
0.75    23.295671
0.80    28.366098
0.85    35.375143
0.90    47.255185
Name: distance, dtype: float64

In [3]:
# coordinate system conversion

df_act= gpd.GeoDataFrame(df_act, geometry=gpd.points_from_xy(x=df_act.POINT_X, y=df_act.POINT_Y), crs = 'EPSG:3006')
df_act = df_act.to_crs(4326)

df_act['lat']=df_act['geometry'].apply(lambda p: p.y)
df_act['lng']=df_act['geometry'].apply(lambda p: p.x)

In [4]:
max_trip_number = df_act.groupby(["person"])["act_id"].max().reset_index()
max_trip_number.rename(columns = {"act_id": "max_trip_number"}, inplace=True)

In [5]:
#do pivot table by activity ID's

df_act= df_act[["person", "act_id", "deso", "act_purpose", "lat", "lng"]]

df_act = df_act.pivot_table(index='person', columns= 'act_id', values=[ "lat", "lng", "deso", "act_purpose"], aggfunc = lambda x: ','.join(str(v) for v in x) )

df_act= pd.merge(df_act, max_trip_number, left_index=True, right_on='person', how='left')
#df_act.columns

In [6]:
df_OTP = pd.DataFrame(columns=['person', 'act_id', 'act_purpose', 'lat_o', 'lng_o', 'lat_d', 'lng_d', 'origin', 'destination'])

for max_trip in range(2, max(max_trip_number.max_trip_number)+1):
    df_something = df_act[df_act.max_trip_number==max_trip]

    for trip in range(max_trip):
        df_something['act_id']=trip
        df_OTP = pd.DataFrame(np.vstack([df_OTP.values, df_something[['person', 'act_id', ('act_purpose',trip+1), ('lat',trip), ('lng',trip), ('lat',trip+1), ('lng',trip+1), ('deso',trip), ('deso',trip+1)]].values]), columns=df_OTP.columns)


In [7]:
df_OTP = df_OTP.rename_axis('MyIdx').sort_values(by = ['person', 'MyIdx'], ascending = [True, True])
df_OTP['origin'] = df_OTP['origin'].str[:4].astype(int)
df_OTP['destination'] = df_OTP['destination'].str[:4].astype(int)
df_OTP[['lat_o', 'lng_o', 'lat_d', 'lng_d']] = df_OTP[['lat_o', 'lng_o', 'lat_d', 'lng_d']].astype(float)

#df_OTP.head()

### calculate an average cycling speed by age,gender and trip purposes for each person

In [8]:
from avg_speed_calculator import main


if __name__ == '__main__':
    df_OTP = main(df_OTP)

In [15]:
df_OTP.tail()

,person,act_id,act_purpose,lat_o,lng_o,lat_d,lng_d,origin,destination,Age,Gender,cycling_speed
818627,10203216,1,other,59.088588,12.790460,59.046517,12.722920,1785,1492,19,Male,25.338978
818628,10203216,2,other,59.046517,12.722920,59.122988,12.889993,1492,1785,19,Male,25.338978
818629,10203216,3,home,59.122988,12.889993,59.043547,12.703662,1785,1492,19,Male,25.338978
818630,10203222,0,home,59.045073,12.719389,59.045881,12.706733,1492,1492,29,Male,16.856775
818631,10203222,1,work,59.045881,12.706733,59.045073,12.719389,1492,1492,29,Male,18.762358


### divede the data for multiprocess routing and save

In [17]:
# divide the data to 30 parts
splits = np.array_split(df_OTP.person, 30)
df_OTP['part'] = 0


In [18]:
count=0
for split in range(30):
    df_OTP['part'][df_OTP.person.isin(splits[split])]=count
    count+=1
    print(count)

# save the data
df_OTP.to_csv(f'./dbs/agents/od_pairs_vg.csv', index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
